<CENTER>
    
    
# MEGALINE #

**Descrição do projeto:**

A operadora de celular Megaline está insatisfeita com o fato de muitos de seus clientes estarem usando planos antigos. A empresa quer desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: **Smart ou Ultra**.

Temos acesso aos dados de comportamento dos clientes que já mudaram para os novos planos. Para essa tarefa de classificação, iremos desenvolver um modelo que escolhe o plano certo. Como os dados já passaram pela etapa de pré-processamento de dados, iremos direto para etapa da criação do modelo.

Iremos desenvolver um modelo com a maior acurácia possível, verificando-a com o conjunto de dados de teste. Neste projeto, o limite mínimo para acurácia é de 0.75

## ETAPAS INICIAIS ##

**Nesta etapa, iremos:**

* importar as bibliotecas necessárias para o trabalho;
* abrir o DataFrame e imprimir as primeiras linhas para termos um panorama geral dos dados;
* verificar as informações gerais do DataFrame.

In [5]:
# Importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
# Abrindo o arquivo de dados e imprimindo as 5 primeiras linhas para avaliação
users_behavior = pd.read_csv('./datasets/users_behavior.csv')

print(users_behavior.head(5))

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [7]:
# Imprimindo as informações do DataFrame para conferência 
print(users_behavior.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


## DIVISÃO DOS DADOS ##

**Nesta etapa dividiremos os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste.**

Como o conjunto de teste não existe, dividiremos em 3 partes da seguinte forma: 80:10:10 para treino, validação e teste respectivamente.

In [8]:
# Definindo as colunas de características e de objetivo
features = users_behavior.drop(['is_ultra'], axis = 1)
target = users_behavior['is_ultra']

# Na primeira etapa, dividiremos os dados em "treinamento" e "conjunto de dados restantes" 
features_train, features_remain, target_train, target_remain = train_test_split(features, target, train_size=0.8, random_state = 14)

# Agora, queremos que os tamanhos dos conjuntos de validação e teste sejam iguais (20% de cada um dos dados gerais) 
# Portanto, vamos definir test_size que represente 50% dos dados restantes (dividir os dados restantes no meio)
features_valid, features_test, target_valid, target_test = train_test_split(features_remain, target_remain, test_size=0.5, random_state = 14)

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(2571, 4)
(2571,)
(321, 4)
(321,)
(322, 4)
(322,)


## ESTUDO DO MELHOR MODELO ##

Nesta etapa, iremos testar vários modelos para avaliar qual deles tem o melhor desempenho com o nosso conjunto de dados. Como estamos lidando com um objetivo que possui dados categóricos, utilizaremos modelos de **classificação**, sendo:

* Árvore de Decisão;
* Floresta Aleatória;
* Regressão Logística

**Para esta avaliação, utilizaremos a acurácia como métrica, mas também precisamos considerar o tempo de execução do código, pois isso incorre em maiores custos computacionais e também interfere na experiência do usuário.**

### Árvore de Decisão: ###

In [9]:
# Usaremos a função time() para medirmos o tempo em segundos para a execução do código.
# Basta registrar o tempo antes e depois da execução do código e calcular a diferença para obter o tempo de execução.

# iniciando o registro de tempo de execução do código
start_tree = time.time()


# criando as variáveis para comparação do melhor modelo de Árvore no ciclo for:
best_score_tree = 0
best_depth = 0
best_criterion = None
best_min_samples_leaf = 0


for criterion in ['gini', 'entropy']: # testando melhor criterio para o modelo
    for leaf in range(1, 11): # testando melhor número de amostras por folha para o modelo
        for depth in range(1, 11): # testando melhor hiperparâmetro de profundidade máxima para o modelo
            model_tree = DecisionTreeClassifier(min_samples_leaf = leaf, max_depth = depth, criterion = criterion, random_state = 14)
            model_tree.fit(features_train, target_train) # treinando o modelo com os dados de treinamento
            score_tree = model_tree.score(features_valid, target_valid) # calculando a Acurácia com os dados de validação
    
        # comparando os modelos para definir os melhores hiperparâmetros
            if score_tree > best_score_tree:
                best_score_tree = score_tree
                best_depth = depth
                best_criterion = criterion
                best_min_samples_leaf = leaf
    
print("Acurácia do melhor modelo de árvore decisória (max_depth = {}, criterion = {} e min_samples_leaf = {}) é: {}"
      .format(best_depth, best_criterion, best_min_samples_leaf, best_score_tree))
print()


# finalizando o registro de tempo de execução do código
end_tree = time.time()

# imprimindo o tempo de execução em segundos
print('Tempo de execução:', end_tree - start_tree, 'segundos')

Acurácia do melhor modelo de árvore decisória (max_depth = 10, criterion = gini e min_samples_leaf = 7) é: 0.8317757009345794

Tempo de execução: 3.1558964252471924 segundos


### Floresta Aleatória: ###

In [10]:
# iniciando o registro de tempo de execução do código com 10 árvores
start_forest = time.time()


# criando as variáveis para comparação do melhor modelo no ciclo for
best_score_forest = 0
best_est = 0
best_bootstrap = None
best_forest_depth = 0


for bootstrap in [True, False]: # verificando qual melhor bootstrap para o modelo
    for forest_depth in range(1, 11): # testando melhor hiperparâmetro de profundidade máxima para o modelo
        for est in range(1, 11): # testando modelos com hiperparâmetro de quantidade de árvores de 1 a 10
            model_forest = RandomForestClassifier(random_state = 14, bootstrap = bootstrap, max_depth = depth, n_estimators = est)
            model_forest.fit(features_train, target_train) # treinando o modelo com os dados de treinamento
            score_forest = model_forest.score(features_valid, target_valid) # calculando a Acurácia com os dados de validação
    
     # comparando os modelos para definir qual a melhor quantidade de árvores 
            if score_forest > best_score_forest:
                best_score_forest = score_forest
                best_est = est
                best_bootstrap = bootstrap
                best_forest_depth = forest_depth
        
print("Acurácia do melhor modelo de floresta aleatória (max_depth = {}, bootstrap = {} e n_estimator = {}) é: {}"
      .format(best_forest_depth, best_bootstrap, best_est, best_score_forest))
print()


# finalizando o registro de tempo de execução do código
end_forest = time.time()

# imprimindo o tempo de execução em segundos
print('Tempo de execução:', end_forest - start_forest, 'segundos')

Acurácia do melhor modelo de floresta aleatória (max_depth = 1, bootstrap = False e n_estimator = 9) é: 0.8317757009345794

Tempo de execução: 10.082597732543945 segundos


### Regressão Logística: ###

In [11]:
# iniciando o registro de tempo de execução do código
start_regression = time.time()

# adicionando aleatoriedade aos algorítmos e definindo solucionador como 'liblinear'
model_regression = LogisticRegression(random_state = 14, solver = 'liblinear') 

# treinando o modelo com os dados de treinamento
model_regression.fit(features_train, target_train)

# calculando a Acurácia com os dados de validação
score_valid_regression = model_regression.score(features_valid, target_valid)
print("Acurácia do conjunto de validação é:", score_valid_regression) 


# finalizando o registro de tempo de execução do código
end_regression = time.time()

# imprimindo o tempo de execução em segundos
print()
print('Tempo de execução:', end_regression - start_regression, 'segundos')

Acurácia do conjunto de validação é: 0.7133956386292835

Tempo de execução: 0.11326837539672852 segundos


### Comparação dos resultados e definição do melhor modelo: ###

Conforme avaliado nos testes realizados, **o melhor modelo para nossos dados foi o de Árvore Decisória**, pois obtivemos uma acurácia satisfatória de 83%, exatamente a mesma obtida com o modelo da Floresta Aleatória, porém, com um tempo de execução menor para a execução do código, sendo este tempo de cerca de 2 segundos, enquanto o da Floresta Aleatória foi de cerca de 7.5 segundos (quase 4 vezes maior).

Portanto, **escolhemos o modelo da Árvore Decisória com os hiperparâmetros (max_depth = 10, criterion = gini e min_samples_leaf = 7)  para seguir**, pois obtivemos um resultado satisfatório tanto em relação à acurácia, quanto em relação ao tempo de execução do modelo.

## TESTE DO MELHOR MODELO: ##

Agora que já escolhemos o modelo mais adequado para seguir, vamos treiná-lo usando todo o conjunto de dados de origem e depois vamos testá-lo com os dados separados para teste.

In [12]:
# utilizando o modelo selecionado anteriormente com os seus hiperparâmetros
best_model = DecisionTreeClassifier(random_state = 14, max_depth = 10, criterion = 'gini', min_samples_leaf = 7)

# treinando o modelo com todos os dados de origem disponíveis
best_model.fit(features, target) 

# calculando a Acurácia do melhor modelo com os dados separados para teste
best_model_score = best_model.score(features_test, target_test)

print("Acurácia do melhor modelo com o conjunto de teste:", best_model_score)

Acurácia do melhor modelo com o conjunto de teste: 0.84472049689441


**O resultado do teste foi bem satisfatório para nosso modelo, pois alcançamos uma acurácia de 84.5%**

## PROVA REAL DO MODELO: ##

Nesta etapa iremos tirar a prova real do modelo.

**Para realizar esta comparação devemos avaliar dois resultados de acurácia:**

* um resultado de acurácia para as predições do modelo com os dados reais;

* um resultado de acurácia para as predições do modelo com valores ao acaso.


Como temos apenas duas categorias (**classificação binária**) a acurácia do modelo em relação aos dados aleatórios deverá ficar em torno dos 50%.

In [13]:
# criando uma coluna com dados aleatórios com a mesma quantidade de linhas do nosso DataFrame
test_predictions = pd.Series(np.random.randint(2, size = 3214))

In [14]:
# calculando as predições do modelo final treinado utilizando somente os dados de características
best_predictions = best_model.predict(features)

# comparando os resultados reais da coluna target com os resultados previstos pelo modelo
best_accuracy = accuracy_score(target, best_predictions)

# comparando os resultados previstos pelo modelo com a coluna de dados aleatórios
test_accuracy = accuracy_score(test_predictions, best_predictions)

print("Acuracidade da previsão do modelo com os dados reais é de:", best_accuracy)
print()
print("Acuracidade da previsão do modelo com os dados aleatórios é de:", test_accuracy)

Acuracidade da previsão do modelo com os dados reais é de: 0.8459863098942129

Acuracidade da previsão do modelo com os dados aleatórios é de: 0.4940883634100809


**Conclusão:**

Os resultados da prova real foram dentro do esperado, pois podemos observar que o modelo teve uma acurácia de aproximadamente 84.5% com os dados reais, enquanto obteve apenas cerca de 50% de acurácia com os dados aleatórios.

Os resultados obtidos nos provam que nosso modelo é eficiente naquilo que se propõe.

<center>

# Renan Rosental de Oliveira #
